In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df1 = pd.read_csv('dataset1.csv')
df2 = pd.read_csv('dataset2.csv')

In [3]:
df1['start_time'] = pd.to_datetime(df1['start_time'], errors='coerce')

/tmp/ipykernel_271/1500980969.py:1: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df1['start_time'] = pd.to_datetime(df1['start_time'], errors='coerce')


In [4]:
df2['time']= pd.to_datetime(df2['time'], format="%d/%m/%Y %H:%M", errors='coerce')
df2['month'] = df2['month'].astype('category')
df2.dtypes

time                  datetime64[ns]
month                       category
hours_after_sunset           float64
bat_landing_number             int64
food_availability            float64
rat_minutes                  float64
rat_arrival_number             int64
dtype: object

In [5]:
df2['time_end'] = df2['time'] + pd.Timedelta(minutes=30)

In [9]:
# Sort both datasets by time to ensure proper matching
df1 = df1.sort_values('start_time')
df2 = df2.sort_values('time')

# Perform a merge_asof to match each landing to the nearest 30-minute period
df_combined = pd.merge_asof(
    df1,
    df2,
    left_on='start_time',
    right_on='time',
    direction='backward',  # Match to the most recent period start
    allow_exact_matches=True
)

# Filter rows where start_time is within the 30-minute window
df_combined = df_combined[
    (df_combined['start_time'] >= df_combined['time']) &
    (df_combined['start_time'] < df_combined['time_end'])
]

df_combined

,start_time,bat_landing_to_food,habit,rat_period_start,rat_period_end,seconds_after_rat_arrival,risk,reward,month_x,sunset_time,hours_after_sunset_x,season,time,month_y,hours_after_sunset_y,bat_landing_number,food_availability,rat_minutes,rat_arrival_number,time_end
0,2017-12-26 20:57:00,1.000000,NaN,26/12/2017 20:53,26/12/2017 20:58,239,0,0,0,26/12/2017 16:43,4.248611,0,2017-12-26 20:43:00,0,4.0,51,2.968874,1.283333,2,2017-12-26 21:13:00
1,2017-12-26 20:57:00,5.000000,NaN,26/12/2017 20:53,26/12/2017 20:58,199,0,0,0,26/12/2017 16:43,4.237500,0,2017-12-26 20:43:00,0,4.0,51,2.968874,1.283333,2,2017-12-26 21:13:00
2,2017-12-26 21:24:00,3.000000,fast,26/12/2017 21:22,26/12/2017 21:27,121,0,1,0,26/12/2017 16:43,4.693611,0,2017-12-26 21:13:00,0,4.5,53,2.561404,4.866667,1,2017-12-26 21:43:00
3,2017-12-26 21:24:00,15.000000,rat,26/12/2017 21:22,26/12/2017 21:27,88,1,0,0,26/12/2017 16:43,4.684444,0,2017-12-26 21:13:00,0,4.5,53,2.561404,4.866667,1,2017-12-26 21:43:00
4,2017-12-26 21:24:00,6.000000,pick,26/12/2017 21:22,26/12/2017 21:27,113,0,1,0,26/12/2017 16:43,4.691389,0,2017-12-26 21:13:00,0,4.5,53,2.561404,4.866667,1,2017-12-26 21:43:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902,2018-05-21 02:54:00,7.000000,NaN,21/05/2018 2:49,21/05/2018 2:57,306,0,0,5,20/05/2018 19:34,7.348611,1,2018-05-21 02:34:00,5,7.0,21,2.000000,7.766667,1,2018-05-21 03:04:00
903,2018-05-21 02:57:00,43.000000,rat_bat_fight,21/05/2018 2:49,21/05/2018 2:57,455,1,0,5,20/05/2018 19:34,7.390000,1,2018-05-21 02:34:00,5,7.0,21,2.000000,7.766667,1,2018-05-21 03:04:00
904,2018-05-21 23:09:00,0.062304,NaN,21/05/2018 23:09,21/05/2018 23:09,15,0,0,5,21/05/2018 19:35,3.574167,1,2018-05-21 23:05:00,5,3.5,41,2.000000,3.616667,2,2018-05-21 23:35:00
905,2018-05-21 23:19:00,83.000000,bat_pick,21/05/2018 23:17,21/05/2018 23:20,116,1,1,5,21/05/2018 19:35,3.736111,1,2018-05-21 23:05:00,5,3.5,41,2.000000,3.616667,2,2018-05-21 23:35:00


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=74c414ef-560d-420d-b4af-809d2b670979' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>